In [1]:
# Import Python Tools:
import pandas as pd
import numpy as np
from numpy import sqrt, pi, average, cos, sin, tan, arcsin, arccos, arctan, deg2rad, rad2deg
from seismic_analysis.ELF.seismic_coefficients import short_period_site_coefficient
from seismic_analysis.ELF.seismic_coefficients import long_period_site_coefficient
from seismic_analysis.ELF.seismic_coefficients import shear_wave_velocity

<div style="page-break-after: always;"></div>

# Seismic Load - Equivalent Lateral Force Method

### Seismic Parameters

In [2]:
risk_category = "III" #I, II, III, IV (this affects the importance factor, Ie)
site_class = "Default" #B, C, D, E, or Default
S_s = 0.267
S_1 = 0.083

V_s30 = shear_wave_velocity(site_class)
print(f"V_s30 = {V_s30} ft/sec")
F_a = short_period_site_coefficient(V_s30, S_s)
if site_class == "Default":
    F_a = max(1.2, F_a)
F_v = long_period_site_coefficient(V_s30, S_1)
print(f"F_a = {round(F_a, 2)}, F_v = {round(F_v, 2)}")

V_s30 = 873 ft/sec
F_a = 1.58, F_v = 2.45


In [3]:
S_MS = S_s * F_a
S_M1 = S_1 * F_v
f"S_MS = {round(S_MS, 3)}g, S_M1 = {round(S_M1, 3)}g"

'S_MS = 0.421g, S_M1 = 0.204g'

In [4]:
S_DS = 2/3 * S_MS
S_D1 = 2/3 * S_M1
T_L = 6 #sec
if risk_category == "III":
    I_e = 1.25
elif risk_category == "IV":
    I_e = 1.50
else:
    I_e = 1.00

R = 3
Ω_0 = 3
C_d = 3
ρ = 1.0
f"I_e = {I_e}, S_DS = {round(S_DS, 3)}g, S_D1 = {round(S_D1, 3)}g"

'I_e = 1.25, S_DS = 0.28g, S_D1 = 0.136g'

### Seismic Response Coefficient, $C_s$

**AISC7-16 Table 12.8-2 Values of Approximate Period Parameters C<sub>t</sub> and x**
| Structure Type | C<sub>t</sub> | x |
|-------------|-----|------------------|
| Steel moment frames | 0.028 | 0.8 |
| Concrete moment frames | 0.016 | 0.9 |
| Steel eccentrically braced frames | 0.03 | 0.75 |
| Steel buckling-restrained braced frames | 0.03 | 0.75 |
| All other structural systems | 0.02 | 0.75 |

* Steel or concrete moment frames: Frames resist 100% of the required seismic force and are not enclosed or adjoined by components that are more rigid and will prevent the frames from deflecting where subjected to seismic forces.
* Steel eccentrically braced frames must be in accordance with Table 12.2-1 lines B1 or D1.

In [5]:
hn = 32 #ft
Ct = 0.02
x = 0.75
T = Ct*hn**x

f"Approximate fundamental period, Ta = {round(T, 3)} sec"

'Approximate fundamental period, Ta = 0.269 sec'

In [6]:
Cs_calc = S_DS / (R/I_e)
print(f"Prelim Cs = {round(Cs_calc, 4)}")

if T <= T_L:
	Cs = min(Cs_calc, S_D1 / (T*R/I_e))
elif T > T_L:
	Cs = min(Cs_calc, S_D1 / (T**2*R/I_e))

print(f"Adjusted Cs = {round(Cs, 4)}")

Prelim Cs = 0.1168
Adjusted Cs = 0.1168


#### Effective Seismic Weight

Effective Seismic Weight = Dead load + ...

* 25% of floor live load for storage areas
* Partition load or minimum of 10 psf if partitions are required by Section 4.3.2
* Total operating weight of permanent equipment
* 20% of uniform design snow load if snow load > 30 psf
* Weight of landscaping and other materials at roof gardens and similar areas

--> storage exception: storage loads are not more than 5% of the effective seismic weight at that level and live load in public garages and open parking structures.

In [7]:
roof_DL = 15 #psf
floor_SIDL = 10 #psf
floor_SW = conc_density * 4.25/12 #psf
wall_DL = 80 #psf
height = hn #ft

bldg_area = (1.5+31.333*4) * (1.5*2+25*3) + 20*(3.333+10.2+3.25+28.75+4.5+1.167) #sf
wall_perimeter = (1.5+31.333*4)*2 + (1.5*2+25*3)*2 #ft

W = (
	roof_DL*bldg_area \
	+ (floor_SIDL + floor_SW)*bldg_area \
	+ wall_DL*wall_perimeter*height \
) / 1000 #kips

f"total seismic weight = {round(W, 2)} kips"

'total seismic weight = 1901.62 kips'

### Base Shear, V

**Seismic Base Shear (ASCE7-16 12.8.1)**
$$ V = C_s W $$

* $V$ = seismic base shear
* $C_s$ = seismic response coefficient
* $W$ = effective seismic weight

In [8]:
V = Cs * W #kips
f"Base Shear, V = {round(V, 2)} kips"

'Base Shear, V = 222.17 kips'

### Vertical Distribution, F<sub>x</sub>

**Vertical Distribution of Seismic Forces (ASCE7-16 12.8.3)**

$$F_x = C_{vx}V$$

$$C_{vx} = \frac{w_x h^k_x}{\displaystyle \sum_{i=1}^{n} w_i h^k_i}$$

* $F_x$ = lateral seismic force induced at any level, kips
* $C_{vx}$ = vertical distribution factor
* $V$ = total base shear, kips
* $w_i$ and $w_x$ = portion of total effective seismic weight of the structure located or assigned to level $i$ or $x$
* $h_i$ and $h_x$ = height from the base to level $i$ or $x$
* $k$ = an exponent related to the structural period:
    - $k$ = 1 for T <= 0.5 s
    - $k$ = 2 for T >= 2.5 s
    - Linear interpolation in between.

In [9]:
h_1 = 15 #ft
h_2 = 32 #ft

In [10]:
h_floor_to_floor = hn / 2 #ft
h_roof_trib = h_floor_to_floor / 2 #ft
w_1 = ((floor_SIDL+floor_SW)*bldg_area + wall_DL*wall_perimeter*h_floor_to_floor) / 1000 #kips

h_roof_trib = (height - h_2)/2
w_2 = (roof_DL*bldg_area + wall_DL*wall_perimeter*h_roof_trib) / 1000 #kips

k = x

print(f"level 2 weight = {round(w_2, 2)} kips")
print(f"leve 1 weight = {round(w_1, 2)} kips")

level 2 weight = 163.75 kips
leve 1 weight = 1213.5 kips


In [11]:
shear_distribution = pd.DataFrame(
    [
    [w_2, h_2, w_2*h_2**k],
    [w_1, h_1, w_1*h_1**k],
    ],
    columns=["w_i", "h_i", "w_i*h_i^k"]
) #table creation

shear_distribution.index = range(len(shear_distribution), 0, -1) #for table display
shear_distribution.index = shear_distribution.index.map(lambda x: f"Level {x}") #for table display

shear_distribution.round(2)

,w_i,h_i,w_i*h_i^k
Level 2,163.75,32,2203.19
Level 1,1213.50,15,9249.28


In [12]:
sum_w_h_k = shear_distribution["w_i*h_i^k"].sum()
sum_w_h_k.round(2)

11452.47

In [13]:
shear_distribution["C_vx"] = shear_distribution["w_i*h_i^k"] / sum_w_h_k
shear_distribution["Fx"] = shear_distribution["C_vx"] * V
shear_distribution.round(2)

,w_i,h_i,w_i*h_i^k,C_vx,Fx
Level 2,163.75,32,2203.19,0.19,42.74
Level 1,1213.50,15,9249.28,0.81,179.43


### Diaphragm Forces

### Story Drift

In [14]:
def seismic_drift(δ_xe):
	δ_x = C_d * δ_xe / I_e
	return δ_x

In [15]:
elastic_story_drifts = [0.1, 0.3, 5] #in (from external calculation)

seismic_story_drifts = []
for δ_xe in elastic_story_drifts:
	δ_x = seismic_drift(δ_xe)
	seismic_story_drifts.append(round(δ_x, 3))
	
f"Seismic Drifts = {seismic_story_drifts} in"

'Seismic Drifts = [0.24, 0.72, 12.0] in'